#Twitter-Bot Detection balanced_dataset

In [11]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install kagglehub pandas matplotlib seaborn tensorflow
%pip install --upgrade tensorflow

In [13]:
%pip install numpy h5py pyyaml protobuf transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\usicc\.cache\kagglehub\datasets\uciml\sms-spam-collection-dataset\versions\1


In [16]:
#Изучить набор данных, исследовать на пропуске, выбросы, балансировку по классам и тд

User ID: Уникальный идентификатор пользователя. Можно использовать для отслеживания уникальных пользователей в анализе.
Username: Имя пользователя. Оно может помочь в дополнительной идентификации или в поиске пользователей в Twitter, если это необходимо.
Tweet: Текст твита. Анализ текстового контента может дать возможность выявить паттерны, типичные для ботов (например, повторяющиеся фразы, отсутствие оригинальности в контенте и т.д.).
Retweet Count: Количество ретвитов. Это может помочь в оценке влияния сообщения и оценке активности пользователей.
Mention Count: Количество упоминаний в твите. Может указать на уровень взаимодействия пользователя с другими аккаунтами.
Follower Count: Количество подписчиков пользователя. Боты часто могут иметь высокий или низкий уровень подписчиков в зависимости от их функциональности.
Verified: Булевое значение, показывающее, подтвержден ли аккаунт. Проверенные аккаунты можно считать более надежными.
Bot Label: Метка, указывающая, является ли пользователь ботом или нет. Эта информация будет полезна для машинного обучения, поскольку она позволяет обучать модели на размеченных данных.
Location: Локация пользователя. Это может помочь в географическом анализе и выявлении паттернов использования ботов в определенных регионах.
Created At: Дата и время создания твита. Это может помочь проанализировать временные паттерны и активность пользователей.
Hashtags: Хэштеги в твите. Анализ хэштегов может помочь понять, о чем говорит пользователь и какие темы наиболее популярны.

In [17]:
import pandas as pd

data = pd.read_csv('spam.csv', encoding='ISO-8859-1')

data = data[['v1', 'v2']]

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
missing_values = data.isnull().sum()
print("Пропущенные значения:\n", missing_values[missing_values > 0])

Пропущенные значения:
 Series([], dtype: int64)


In [19]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

X = data['v2'].values.reshape(-1, 1)
y = data['v1']                        

X_resampled, y_resampled = ros.fit_resample(X, y)

balanced_data = pd.DataFrame(X_resampled, columns=['v2'])
balanced_data['v1'] = y_resampled

print("Class counts after oversampling:\n", balanced_data['v1'].value_counts())

Class counts after oversampling:
 v1
ham     4825
spam    4825
Name: count, dtype: int64


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data['v1'] = data['v1'].astype(str)

data['v1'] = data['v1'].map({'ham': 0, 'spam': 1})

data = data.dropna(subset=['v1', 'v2'])

X_text = data['v2'].values.tolist()
y = data['v1'].values

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

max_length = 200
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))

model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train_padded,
    y_train,
    validation_split=0.2,
    epochs=10, 
    batch_size=512
)

test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Epoch 1/10


c:\Users\usicc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 775ms/step - accuracy: 0.7031 - loss: 0.5093 - val_accuracy: 0.8621 - val_loss: 0.4733
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 698ms/step - accuracy: 0.8724 - loss: 0.3725 - val_accuracy: 0.8621 - val_loss: 0.4964
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 705ms/step - accuracy: 0.8696 - loss: 0.3497 - val_accuracy: 0.8621 - val_loss: 0.4597
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 720ms/step - accuracy: 0.8776 - loss: 0.2640 - val_accuracy: 0.9776 - val_loss: 0.3557
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 729ms/step - accuracy: 0.9711 - loss: 0.1400 - val_accuracy: 0.9787 - val_loss: 0.2521
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 741ms/step - accuracy: 0.9846 - loss: 0.0767 - val_accuracy: 0.9821 - val_loss: 0.1768
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 764ms/step - accuracy: 0.9916 - loss: 0.0430 - val_accuracy: 0.9798 - val_loss: 0.1517
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 764ms/step - accuracy: 0.9942 - loss: 0.0310 - val_accuracy: 0.9854 - val_loss: 0.1297
Epo